# MNIST with Tensorboard

Last update: 2022-07-20, Tuan Do

Example of ML experiment with MNIST with tensorboard. We will assume that tensorboard is already running in the background or will be run after this notebook. See other instructions for how to create a local instance of tensorboard or upload to tensorboard dev.

In [7]:
import tensorflow as tf
import datetime
import tensorboard
print(tensorboard.__version__)
print(tensorboard.__path__)

2.5.0
['/home/tdo/miniconda3/envs/jupyterhub3/lib/python3.8/site-packages/tensorboard']


In [8]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
  ])

In [9]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# location where we will save our logs. This should also be the location that tensorboard watches. 
# In this case to not overwrite the logs, we will name our experiment after the date and time it was started
log_dir = "/logs/mnist_cnn_v1/"
epochs = 10

# use this to log into tensorboard and look at performance in real time
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2186 - accuracy: 0.9356 - val_loss: 0.1111 - val_accuracy: 0.9651
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0969 - accuracy: 0.9712 - val_loss: 0.0803 - val_accuracy: 0.9746
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0686 - accuracy: 0.9781 - val_loss: 0.0768 - val_accuracy: 0.9779
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0526 - accuracy: 0.9831 - val_loss: 0.0715 - val_accuracy: 0.9764
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0427 - accuracy: 0.9859 - val_loss: 0.0678 - val_accuracy: 0.9801
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0362 - accuracy: 0.9878 - val_loss: 0.0702 - val_accuracy: 0.9808
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0312 - accuracy: 0.9899 - val_loss: 0.0717 - val_accuracy:

In [10]:
# list what is in the log files
!ls $log_dir

train  validation
